In [1]:
import org.apache.spark.rdd.RDD
import org.apache.spark.ml.feature.{Word2Vec, Word2VecModel}

Intitializing Scala interpreter ...

Spark Web UI available at http://VIEANTH.mshome.net:4040
SparkContext available as 'sc' (version = 2.4.3, master = local[*], app id = local-1562947555367)
SparkSession available as 'spark'


import org.apache.spark.rdd.RDD
import org.apache.spark.ml.feature.{Word2Vec, Word2VecModel}


In [2]:
def printRDD[T] ( rdd:RDD[T], n:Int = 0 ) : Unit = {
    if(n != 0) {
    rdd.take(n).foreach(println)
} else {
    rdd.collect().foreach(println)}
}

printRDD: [T](rdd: org.apache.spark.rdd.RDD[T], n: Int)Unit


In [3]:
var data = sc.textFile("data/item_metadata.csv")

data: org.apache.spark.rdd.RDD[String] = data/item_metadata.csv MapPartitionsRDD[1] at textFile at <console>:27


In [4]:
printRDD(data, 10)

item_id,properties
5101,Satellite TV|Golf Course|Airport Shuttle|Cosmetic Mirror|Safe (Hotel)|Telephone|Hotel|Sitting Area (Rooms)|Reception (24/7)|Air Conditioning|Hypoallergenic Rooms|Cable TV|Hotel Bar|Pool Table|Bathtub|Satisfactory Rating|Room Service|Luxury Hotel|Terrace (Hotel)|Television|Minigolf|Business Hotel|Shower|Cot|Gym|Hairdryer|Hypoallergenic Bedding|Accessible Parking|From 3 Stars|Good Rating|Radio|4 Star|From 4 Stars|Family Friendly|Desk|Tennis Court (Indoor)|Balcony|WiFi (Public Areas)|Openable Windows|Express Check-In / Check-Out|Restaurant|Laundry Service|Ironing Board|Tennis Court|From 2 Stars|Business Centre|Bowling|Conference Rooms|Electric Kettle|Accessible Hotel|Porter|Bike Rental|Non-Smoking Rooms|Car Park|Safe (Rooms)|Fitness|Fan|Flatscreen TV|Computer with Internet|WiFi (Rooms)|Lift|Central Heating
5416,Satellite TV|Cosmetic Mirror|Safe (Hotel)|Telephone|Hotel|Sitting Area (Rooms)|Reception (24/7)|Wheelchair Accessible|Hypoallergenic Rooms|Hotel Bar|Bathtub

In [5]:
val header = data.first()
data = data.filter(row => row != header)

header: String = item_id,properties
data: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[2] at filter at <console>:29


In [6]:
val sampled_data = data.sample(false, 0.00001, 42)

sampled_data: org.apache.spark.rdd.RDD[String] = PartitionwiseSampledRDD[3] at sample at <console>:28


In [7]:
sampled_data.count()

res1: Long = 10


In [8]:
val truncated_data = data.map(_.split(',')(1))

truncated_data: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[4] at map at <console>:28


In [9]:
val preprocessed_Data = truncated_data.map(_.split('|')).map(Tuple1.apply).toDF("text")

preprocessed_Data: org.apache.spark.sql.DataFrame = [text: array<string>]


In [10]:
val words = truncated_data.flatMap(_.split('|')).distinct()

words: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[10] at distinct at <console>:28


In [11]:
printRDD(words, 10)
words.count()

Direct beach access
Guest House
Ironing Board
Bowling
Towels
Horse Riding
Reception (24/7)
Cot
2 Star
Cosmetic Mirror


res2: Long = 157


In [12]:
preprocessed_Data.head(1)

res3: Array[org.apache.spark.sql.Row] = Array([WrappedArray(Satellite TV, Golf Course, Airport Shuttle, Cosmetic Mirror, Safe (Hotel), Telephone, Hotel, Sitting Area (Rooms), Reception (24/7), Air Conditioning, Hypoallergenic Rooms, Cable TV, Hotel Bar, Pool Table, Bathtub, Satisfactory Rating, Room Service, Luxury Hotel, Terrace (Hotel), Television, Minigolf, Business Hotel, Shower, Cot, Gym, Hairdryer, Hypoallergenic Bedding, Accessible Parking, From 3 Stars, Good Rating, Radio, 4 Star, From 4 Stars, Family Friendly, Desk, Tennis Court (Indoor), Balcony, WiFi (Public Areas), Openable Windows, Express Check-In / Check-Out, Restaurant, Laundry Service, Ironing Board, Tennis Court, From 2 Stars, Business Centre, Bowling, Conference Rooms, Electric Kettle, Accessible Hotel, Porter, Bike R...

In [13]:
var word2vec = new Word2Vec()
word2vec.setInputCol("text")

word2vec: org.apache.spark.ml.feature.Word2Vec = w2v_128fd52ca557
res4: org.apache.spark.ml.feature.Word2Vec = w2v_128fd52ca557


In [14]:
word2vec.getDefault(word2vec.getParam("maxIter"))
word2vec.getDefault(word2vec.getParam("maxSentenceLength"))
word2vec.getDefault(word2vec.getParam("minCount"))
word2vec.getDefault(word2vec.getParam("numPartitions"))
word2vec.getDefault(word2vec.getParam("vectorSize"))

res5: Option[Any] = Some(100)


In [15]:
val model = word2vec.fit(preprocessed_Data)

model: org.apache.spark.ml.feature.Word2VecModel = w2v_128fd52ca557


In [18]:
def getVectors(model:Word2VecModel) : Map[String, Array[Float]] = {
    var map = Map[String, Array[Float]]()
    for(x <- words) {
        map ++ x -> model.getVectors(x)
    }
    return map
}

getVectors: (model: org.apache.spark.ml.feature.Word2VecModel)Map[String,Array[Float]]


In [35]:
var x = model.getVectors("vector")

x: org.apache.spark.sql.Column = vector


In [21]:
model.findSynonyms("Gym", 5).take(5)

res8: Array[org.apache.spark.sql.Row] = Array([Spa Hotel,0.6347548365592957], [Nightclub,0.5499850511550903], [Beauty Salon,0.5434030294418335], [Ski Resort,0.5368881821632385], [Hypoallergenic Bedding,0.53089439868927])
